In [ ]:
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
xTModell = xthreat.load_model(
    "/home/morten/Develop/packing-report/xT-impact/models/xT_test"
)

In [ ]:
choosen_match_id = 1557709

In [ ]:
df_games

In [ ]:
ws = sd.WhoScored(
    leagues="GER-Bundesliga2",
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

loader = ws.read_events(match_id=choosen_match_id, output_fmt="loader")

In [ ]:
df_actions_ltr = spadl.play_left_to_right(
    df_actions, df_games[df_games["game_id"] == choosen_match_id].home_team_id.values[0]
)
df_shot_actions = df_actions_ltr[df_actions_ltr["type_id"] == 11]
xG_ratings = xTModell.rate_xG(df_shot_actions.reset_index())
df_xg = pd.DataFrame(
    {
        "time": ((df_shot_actions["period_id"] - 1) * 45)
        + (df_shot_actions["time_seconds"] / 60),
        "team": df_shot_actions["team_id"],
        "xg": xG_ratings,
    }
)
# GI actions
df_goals = df_shot_actions[df_shot_actions["result_id"] == 1]
df_own_goals = df_actions_ltr[df_actions_ltr["result_id"] == 3]
df_all_goals = pd.concat([df_goals, df_own_goals], ignore_index=True)
df_all_goals2 = pd.DataFrame(
    {
        "time": ((df_all_goals["period_id"] - 1) * 45)
        + (df_all_goals["time_seconds"] / 60),
        "team_id": [
            x["team_id"]
            if (x["result_id"] == 1)
            else df_teams[df_teams["team_id"] != x["team_id"]]["team_id"].values[0]
            for _, x in df_all_goals.iterrows()
        ],
    }
)

In [ ]:
df_all_goals2

In [ ]:
df_all_goals.loc[
    (df_all_goals["team"] == 1143)
    & (df_all_goals["time"] < 45)
    & (df_all_goals["xt"] > 0)
][
    "xt"
].sum()  # all

In [ ]:
df_teams = loader.teams(game_id=choosen_match_id)
df_teams

In [ ]:
df_players = loader.players(game_id=choosen_match_id)
df_players

In [ ]:
df_games = loader.games(competition_id="GER-Bundesliga2", season_id="2122")
df_games

In [ ]:
df_events = loader.events(game_id=choosen_match_id)
df_events

In [ ]:
ce = sd.ClubElo(
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
)

In [ ]:
df_actions = (
    spadl.opta.convert_to_actions(
        df_events,
        df_games[df_games["game_id"] == choosen_match_id].home_team_id.values[0],
    )
    .merge(spadl.actiontypes_df())
    .merge(df_players[["player_name", "player_id"]])
)
df_actions_ltr = spadl.play_left_to_right(
    df_actions, df_games[df_games["game_id"] == choosen_match_id].home_team_id.values[0]
)
df_actions_ltr

In [ ]:
df_succ = xthreat.get_successful_move_actions(df_actions_ltr)
df_succ

In [ ]:
xt = xTModell.rate(df_succ)
xt_frame = pd.DataFrame(
    {
        "time": ((df_succ["period_id"] - 1) * 45) + (df_succ["time_seconds"] / 60),
        "team": df_succ["team_id"],
        "xt": xt,
    }
)

In [ ]:
xt_frame.loc[(xt_frame["team"] == 1143) & (xt_frame["time"] < 100)]["xt"].sum()  # all

In [ ]:
xt_frame.loc[
    (xt_frame["team"] == 1143) & (xt_frame["time"] < 100) & (xt_frame["xt"] > 0)
][
    "xt"
].sum()  # all

In [ ]:
for idx, game_id in enumerate(tqdm(df_games["game_id"])):
    df_events = loader.events(game_id=game_id)
    df_actions = (
        spadl.opta.convert_to_actions(
            df_events, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
        )
        .merge(spadl.actiontypes_df())
        .merge(df_players[["player_name", "player_id"]])
    )
    df_actions_ltr = spadl.play_left_to_right(
        df_actions, df_games[df_games["game_id"] == game_id].home_team_id.values[0]
    )
    actions_ltr = succ_actions(game_id, df_events, df_games)
    succs_events = xthreat.get_successful_move_actions(actions_ltr)
    xt_ratings = xTModell.rate(succs_events)
    df_players = loader.players(game_id=game_id)
    fill_ids(succs_events, df_players)
    player_values = reset_player_xt_values(succs_events)
    for indx, y in enumerate(succs_events["player_id"]):
        player_values[y] += xt_ratings[indx]
    for x in player_values.keys():
        if player_values[x] < 0:
            player_values[x] = 0
    team_values = {}
    df_teams = loader.teams(game_id=game_id)
    ## reset team values
    for x in df_teams["team_id"]:
        team_values[x] = 0
    for x in player_values.keys():
        team_values[
            df_players[df_players["player_id"] == x]["team_id"].values[0]
        ] += player_values[x]
    game = df_games[df_games["game_id"] == game_id]
    game_date = np.datetime_as_string(game["game_date"].values[0])[:10]
    game_comp_id = game["competition_id"].values[0]
    ce_df = ce.read_by_date(game_date)
    league_mean = np.mean(ce_df[ce_df["league"] == game_comp_id].elo)
    for x in player_values.keys():
        team_name = df_teams[
            df_teams["team_id"]
            != df_players[df_players["player_id"] == x]["team_id"].values[0]
        ]["team_name"].values[0]
        team_elo = ce_df.loc[team_name].elo
        id_to_xt_impact[x] += player_values[x] * (
            player_values[x]
            / team_values[df_players[df_players["player_id"] == x]["team_id"].values[0]]
        )
        id_to_xt_impact_elo[x] += (
            player_values[x]
            * (
                player_values[x]
                / team_values[
                    df_players[df_players["player_id"] == x]["team_id"].values[0]
                ]
            )
        ) * (team_elo / league_mean)

In [ ]:
df_res = pd.DataFrame.from_dict(
    {
        "ids": [x for x in id_to_name.keys()],
        "names": [id_to_name[x] for x in id_to_name.keys()],
        "impact": [id_to_xt_impact[x] for x in id_to_name.keys()],
        "impact_elo": [id_to_xt_impact_elo[x] for x in id_to_name.keys()],
    }
)

In [ ]:
df_res.sort_values("impact", ascending=False).head(20)

In [ ]:
id_to_name, id_to_xt_impact, id_to_xt_impact_elo = {}, {}, {}

for idx, game_id in enumerate(tqdm(df_games["game_id"])):
    df_events = loader.events(game_id=game_id)
    actions_ltr = succ_actions(game_id, df_events, df_games)
    succs_events = xthreat.get_successful_move_actions(actions_ltr)
    xt_ratings = xTModell.rate(succs_events)
    df_players = loader.players(game_id=game_id)
    # fill_ids(succs_events, df_players)
    for y in succs_events["player_id"]:
        if y not in id_to_name:
            player_name = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_name[y] = df_players[df_players["player_id"] == y][
                "player_name"
            ].values[0]
            id_to_xt_impact[y] = 0
            id_to_xt_impact_elo[y] = [[], [], []]
    player_values = reset_player_xt_values(succs_events)
    for indx, y in enumerate(succs_events["player_id"]):
        player_values[y] += xt_ratings[indx]
    for x in player_values.keys():
        if player_values[x] < 0:
            player_values[x] = 0
    team_values = {}
    df_teams = loader.teams(game_id=game_id)
    ## reset team values
    for x in df_teams["team_id"]:
        team_values[x] = 0
    for x in player_values.keys():
        team_values[
            df_players[df_players["player_id"] == x]["team_id"].values[0]
        ] += player_values[x]
    game = df_games[df_games["game_id"] == game_id]
    game_date = np.datetime_as_string(game["game_date"].values[0])[:10]
    game_comp_id = game["competition_id"].values[0]
    ce_df = ce.read_by_date(game_date)
    league_mean = np.mean(ce_df[ce_df["league"] == game_comp_id].elo)
    for x in player_values.keys():
        if df_players[df_players["player_id"] == x]["is_starter"].values[0]:
            team_name = df_teams[
                df_teams["team_id"]
                != df_players[df_players["player_id"] == x]["team_id"].values[0]
            ]["team_name"].values[0]
            team_elo = ce_df.loc[team_name].elo
            id_to_xt_impact[x] += player_values[x] * (
                player_values[x]
                / team_values[
                    df_players[df_players["player_id"] == x]["team_id"].values[0]
                ]
            )
            id_to_xt_impact_elo[x][1].append(
                (
                    player_values[x]
                    * (
                        player_values[x]
                        / team_values[
                            df_players[df_players["player_id"] == x]["team_id"].values[
                                0
                            ]
                        ]
                    )
                )
                * (team_elo / league_mean)
            )
            id_to_xt_impact_elo[x][0].append(
                df_players[df_players["player_id"] == x]["minutes_played"].values[0]
            )
            id_to_xt_impact_elo[x][2].append(
                id_to_xt_impact_elo[x][1][-1] / id_to_xt_impact_elo[x][0][-1]
            )